In [1]:
import config
import os
import cv2
import albumentations as A
import numpy as np
from tqdm import tqdm


In [ ]:

# Parâmetros
N = 20
target_size = (224, 224)

# Diretórios
medetec_path     = config.medetec_dataset_path
fuseg_path       = config.fuseg_dataset_path

output_train_image_dir = os.path.join(medetec_path, 'train_aug/images')
output_train_mask_dir  = os.path.join(medetec_path, 'train_aug/labels')

output_validation_image_dir = os.path.join(medetec_path, 'validation_aug/images')
output_validation_mask_dir  = os.path.join(medetec_path, 'validation_aug/labels')

output_test_image_dir = os.path.join(medetec_path, 'test_aug/images')
output_test_mask_dir  = os.path.join(medetec_path, 'test_aug/labels')

output_fuseg_image_dir = os.path.join(fuseg_path, 'train_aug/images')
output_fuseg_mask_dir  = os.path.join(fuseg_path, 'train_aug/labels')


os.makedirs(output_train_image_dir, exist_ok=True)
os.makedirs(output_train_mask_dir, exist_ok=True)
os.makedirs(output_validation_image_dir, exist_ok=True)
os.makedirs(output_validation_mask_dir, exist_ok=True)
os.makedirs(output_test_image_dir, exist_ok=True)
os.makedirs(output_test_mask_dir, exist_ok=True)
os.makedirs(output_fuseg_image_dir, exist_ok=True)
os.makedirs(output_fuseg_mask_dir, exist_ok=True)

# Transformações de aumento
transform = A.Compose([
    A.Resize(*target_size),  # garante tamanho 224x224 antes de qualquer augment
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.7, border_mode=cv2.BORDER_REFLECT),
    A.RandomBrightnessContrast(p=0.5),
    A.ElasticTransform(p=0.2),
    A.GaussianBlur(p=0.3),
    A.GridDistortion(p=0.2),
    #A.GaussNoise(p=0.2),
])

### --- Função para processar um dataset ---
def augment_dataset(image_dir, mask_dir, output_image_dir, output_mask_dir, prefix, fix_mask_func=None):
    image_filenames = sorted(os.listdir(image_dir))
    for img_name in tqdm(image_filenames, desc=f"Augmentando {prefix}"):
        img_path = os.path.join(image_dir, img_name)
        mask_path = os.path.join(mask_dir, img_name)

        image = cv2.imread(img_path)
        mask  = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if fix_mask_func:
            mask = fix_mask_func(mask)

        base = f"{prefix}_{img_name[:-4]}"

        # Salvar original (redimensionado)
        orig = transform(image=image, mask=mask)
        cv2.imwrite(os.path.join(output_image_dir, f"{base}_orig.png"), orig['image'])
        cv2.imwrite(os.path.join(output_mask_dir, f"{base}_orig.png"), orig['mask'])

        # N aumentações
        for i in range(N):
            aug = transform(image=image, mask=mask)
            cv2.imwrite(os.path.join(output_image_dir, f"{base}_aug{i}.png"), aug['image'])
            cv2.imwrite(os.path.join(output_mask_dir, f"{base}_aug{i}.png"), aug['mask'])


### --- Correção para máscaras do Medetec ---
def fix_mask_medetec(mask):
    return (mask > 0).astype(np.uint8) * 255


### --- Aplicar para cada dataset ---
# augment_dataset(
#     image_dir=os.path.join(medetec_path, 'validation/images'),
#     mask_dir=os.path.join(medetec_path, 'validation/labels'),
#     output_image_dir=output_validation_image_dir, 
#     output_mask_dir=output_validation_mask_dir,
#     prefix='medetec',
#     fix_mask_func=fix_mask_medetec
# )

# augment_dataset(
#     image_dir=os.path.join(medetec_path, 'test/images'),
#     mask_dir=os.path.join(medetec_path, 'test/labels'),
#     output_image_dir=output_test_image_dir, 
#     output_mask_dir=output_test_mask_dir,
#     prefix='medetec',
#     fix_mask_func=fix_mask_medetec
# )

# augment_dataset(
#     image_dir=os.path.join(medetec_path, 'train/images'),
#     mask_dir=os.path.join(medetec_path, 'train/labels'),
#     output_image_dir=output_train_image_dir, 
#     output_mask_dir=output_train_mask_dir,
#     prefix='medetec',
#     fix_mask_func=fix_mask_medetec
# )

augment_dataset(
    image_dir=os.path.join(fuseg_path, 'train/images'),
    mask_dir=os.path.join(fuseg_path, 'train/labels'),
    output_image_dir=output_fuseg_image_dir, 
    output_mask_dir=output_fuseg_mask_dir,
    prefix='fuseg',
    fix_mask_func=None  # já estão em escala de cinza apropriada
)


Augmentando medetec: 100%|██████████| 30/30 [00:03<00:00,  8.70it/s]
